<a href="https://colab.research.google.com/github/SalmenzarZV/red_neuronal_b-sica/blob/main/red_neuronal_b%C3%A1sica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Implementación de la red neuronal con retropropagación

In [ ]:
class NetNode(object):

  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

In [ ]:
class Network(object):

  def __init__(self, layers):
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    sizes = len(layers)
    for layer in range(1, sizes):
        for node in self.net[layer]:
            for unit in self.net[layer - 1]:
                node.inputs.append(unit)
                node.weights.append(0)

  def accuracy(self, examples):
    correct = 0
    for x_test, y_test in examples:
        prediction = self.predict(x_test)
        if (y_test[prediction] == 1):
            correct += 1
    return correct / len(examples)

  def backpropagation(self, eta, examples, epochs):
    inputs = self.net[0]
    outputs = self.net[-1]
    layer_size = len(self.net)
    for layer in self.net[1:]:
        for node in layer:
            node.weights = [np.random.uniform() for _ in range(len(node.weights))]
    for epoch in range(epochs):
        for x_train, y_train in examples:
            for value, node in zip(x_train, inputs):
                node.value = value
            for layer in self.net[1:]:
                for node in layer:
                    in_val = [n.value for n in node.inputs]
                    unit_value = np.dot(in_val, node.weights)
                    node.value = self.relu(unit_value)
            delta = [[] for _ in range(layer_size)]
            err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
            delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            hidden_layers = layer_size - 2
            for i in range(hidden_layers, 0, -1):
                layer = self.net[i]
                n_layers = len(layer)
                w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
            for i in range(1, layer_size):
                layer = self.net[i]
                in_val = [node.value for node in self.net[i - 1]]
                n_layers = len(self.net[i])
                for j in range(n_layers):
                    layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
        print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
  
  def predict(self, input_data):
    inputs = self.net[0]
    for v, n in zip(input_data, inputs):
        n.value = v
    for layer in self.net[1:]:
        for node in layer:
            in_val = [n.value for n in node.inputs]
            unit_value = np.dot(in_val, node.weights)
            node.value = self.relu(unit_value)
    outputs = self.net[-1]
    return outputs.index(max(outputs, key=lambda node: node.value))

  def relu(self, z):
    return max(0, z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
    pass

  def sigmoide_prime(self, z):
    pass
  
  def set_weights(self, weight_list):
    for layer_idx, layer_weights in enumerate(weight_list):
        layer_nodes = self.net[layer_idx]
        if len(layer_weights) != len(layer_nodes):
            raise ValueError("La longitud de la lista de pesos no coincide con el número de nodos en la capa correspondiente")
        for node_idx, node_weights in enumerate(layer_weights):
            node = layer_nodes[node_idx]
            if len(node_weights) != len(node.inputs):
                raise ValueError("La longitud de la lista de pesos para un nodo no coincide con el número de entradas del nodo")
            node.weights = node_weights

  def weights(self):
    weights = []
    layer_i = 1
    for layer in self.net:
      layer_dict = {'i': layer_i, 'weights': []}
      for node in layer:
        layer_dict['weights'].append(node.weights)
      weights.append(layer_dict)
      layer_i += 1
    return weights


# Usando la red neuronal con un dataset

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [ ]:
iris_x_normalized = normalize(iris_X, axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

In [ ]:
examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

In [ ]:
net = Network([4, 7, 3])
net.backpropagation(0.1, examples, 500)

epoch 0/500 | total error=0.00010486881719447421
epoch 1/500 | total error=0.0002942796184738713
epoch 2/500 | total error=0.0002781090299421501
epoch 3/500 | total error=0.0002503368208590021
epoch 4/500 | total error=0.00021622385704528948
epoch 5/500 | total error=0.0001739798967246543
epoch 6/500 | total error=0.00012340300662554705
epoch 7/500 | total error=6.709250201806839e-05
epoch 8/500 | total error=1.5600029896060037e-05
epoch 9/500 | total error=4.1105006732961025e-05
epoch 10/500 | total error=0.00019408853857758498
epoch 11/500 | total error=0.0004268107886922061
epoch 12/500 | total error=0.0007028309529813583
epoch 13/500 | total error=0.0009930813650509296
epoch 14/500 | total error=0.0012678975138223794
epoch 15/500 | total error=0.0015326614213395254
epoch 16/500 | total error=0.0017680673439837346
epoch 17/500 | total error=0.0019907359227257383
epoch 18/500 | total error=0.002207646976557265
epoch 19/500 | total error=0.0023943893712535264
epoch 20/500 | total erro

In [ ]:
#precisión alcanzada con los datos de entrenamiento
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.95


In [ ]:
#precisión alcanzada con los datos de prueba
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9666666666666667


In [ ]:
#probando con un dato
prediction = net.predict(X_test[2])
print(f"Desired output: {y_test[2]}")
print(f"Index of output: {prediction}")

Desired output: [0. 1. 0.]
Index of output: 1


# Modificación de la implementación de la red neuronal

In [ ]:
# agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas

# agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas
weights = [[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]]
net.set_weights(weights)
net.weights()

ValueError: ignored

In [ ]:
# agrega los métodos sigmoide() y sigmoide_prime() a la clase Network
# modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()

In [ ]:
# los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación
# modifícalos de tal manera que también se puedan utilizar con problemas de regresión

In [ ]:
# modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento

In [ ]:
# una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])

In [ ]:
# ejecuta la red neuronal para los datos de ejemplo de los apuntes
# comprueba los valores de los nodos y de los pesos
# los valores de los nodos tienen que ser los mismos que los de los apuntes
# los valores de los pesos son ligeramente diferentes, ¿por qué?

net = Network([3, 4, 2])

net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# o
net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

# [{'i': 1, 'weights': [[], [], [], []]},
#  {'i': 2,
#   'weights': [array([ 1.3461928 , -0.20382956,  0.92686153, -0.22619916]),
#    array([ 2.81815556,  3.45870697, -2.42784403, -4.61452826]),
#    array([ 3.32877543,  0.76279486, -0.77602416, -0.89576795]),
#    array([ 0.08577962, -0.51118717,  1.05240335,  0.00770549]),
#    array([ 0.22034583, -1.28195713,  2.09186971,  0.75607245]),
#    array([ 4.48121102,  2.15951458, -2.53784711, -2.99235527]),
#    array([-4.45996095, -5.23265064,  5.99733189,  2.81197177])]},
#  {'i': 3,
#   'weights': [array([ 0.07446002,  1.26010194,  0.93825058, -0.50205333, -1.92027562,
#            0.6060396 , -2.08609631]),
#    array([ 1.3055382 , -4.75078351,  2.34340205,  0.35292303,  0.62075582,
#            1.89077643, -6.87860432]),
#    array([ 1.19113633, -4.58842361, -0.64149014,  0.9072472 ,  1.8189326 ,
#           -4.75956505,  1.86726301])]}]

valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
net.weights()

TypeError: ignored